In [7]:
# This file is for parsing the input
def check_for_comma(column_data):
    return column_data.str.contains(',').any()

def input_parser(data_file):
    data_file = data_file.astype(str)
    cols_with_comma = [
        column for column in data_file.columns if check_for_comma(data_file[column])]

    for column in cols_with_comma:
        data_file[column] = data_file[column].str.split(
            ',').apply(lambda items: [element.strip() for element in items])

    return data_file


In [8]:
# This file is used for all the normalizations
import pandas as pd
from itertools import combinations
import re

def is_list_or_set(item):
    return isinstance(item, (list, set))


def is_superkey(relation, left_hand_side):
    grouped = relation.groupby(
        list(left_hand_side)).size().reset_index(name='count')
    return not any(grouped['count'] > 1)


def powerset(s):
    x = len(s)
    for i in range(1 << x):
        yield [s[j] for j in range(x) if (i & (1 << j)) > 0]


def bcnf_decomposition(relation, dependencies):
    for left_hand_side, right_hand_sides in dependencies.items():
        if set(left_hand_side).issubset(relation.columns) and not is_superkey(relation, left_hand_side):
            right_hand_side_cols = list(left_hand_side) + right_hand_sides
            new_relation1 = relation[right_hand_side_cols].drop_duplicates()
            remaining_cols = list(set(relation.columns) - set(right_hand_sides))
            new_relation2 = relation[remaining_cols].drop_duplicates()
            return [new_relation1, new_relation2]
    return [relation]


def check_1nf(relation):
    if relation.empty:
        return False

    for column in relation.columns:
        unique_types = relation[column].apply(type).nunique()
        if unique_types > 1:
            return False
        if relation[column].apply(lambda x: isinstance(x, (list, dict, set))).any():
            return False

    return True


def check_2nf(primary_key, dependencies):
    partial_dependencies_not_found = True
    for left_hand_side, right_hand_side in dependencies.items():
        if set(left_hand_side).issubset(primary_key) and set(left_hand_side) != set(primary_key):
            partial_dependencies_not_found = False
            break

    return partial_dependencies_not_found


def check_3nf(relations, dependencies):
    for relation in relations:
        attributes = set(relations[relation].columns)
        non_prime_attributes = attributes - set(relation)
        for left_hand_side, right_hand_sides in dependencies.items():
            if all(attr in non_prime_attributes for attr in left_hand_side):
                for right_hand_side in right_hand_sides:
                    if right_hand_side in non_prime_attributes:
                        return False
    return True


def check_bcnf(relations, primary_key, dependencies):
    for relation in relations:
        for left_hand_side, right_hand_sides in dependencies.items():
            if set(left_hand_side).issubset(relation.columns):
                if not is_superkey(relation, left_hand_side):
                    return False
    return True


def check_4nf(relations, mvd_dependencies):
    for relation in relations:
        for left_hand_side, right_hand_sides in mvd_dependencies.items():
            for right_hand_side in right_hand_sides:
                if isinstance(left_hand_side, tuple):
                    left_hand_side_cols = list(left_hand_side)
                else:
                    left_hand_side_cols = [left_hand_side]

                if all(col in relation.columns for col in left_hand_side_cols + [right_hand_side]):
                    grouped = relation.groupby(left_hand_side_cols)[
                        right_hand_side].apply(set).reset_index()
                    if len(grouped) < len(relation):
                        print(
                            f"Multi-valued dependency violation: {left_hand_side} ->-> {right_hand_side}")
                        return False
    return True


def check_5nf(relations):
    i = 0
    candidate_keys_dict = {}
    for relation in relations:
        print(relation)
        user_input = input("Enter the candidate keys:")
        print('\n')
        tuples = re.findall(r'\((.*?)\)', user_input)
        candidate_keys = [tuple(map(str.strip, t.split(','))) for t in tuples]
        candidate_keys_dict[i] = candidate_keys
        i += 1

    print(f'Candidate Keys for tables:')
    print(candidate_keys_dict)
    print('\n')

    j = 0
    for relation in relations:
        candidate_keys = candidate_keys_dict[j]
        j += 1

        data_tuples = [tuple(row) for row in relation.to_numpy()]

        def project(data, attributes):
            return {tuple(row[attr] for attr in attributes) for row in data}

        # Function to check if a set of attributes is a superkey
        def is_superkey(attributes):
            for key in candidate_keys:
                if set(key).issubset(attributes):
                    return True
            return False, candidate_keys_dict

        for i in range(1, len(relation.columns)):
            for attrs in combinations(relation.columns, i):
                if is_superkey(attrs):
                    continue

                projected_data = project(data_tuples, attrs)
                complement_attrs = set(relation.columns) - set(attrs)
                complement_data = project(data_tuples, complement_attrs)

                joined_data = {(row1 + row2)
                               for row1 in projected_data for row2 in complement_data}
                if set(data_tuples) != joined_data:
                    print("Failed 5NF check for attributes:", attrs)
                    return False, candidate_keys_dict

    return True, candidate_keys_dict


def validate_first_nf(relation):
    flag_1nf = check_1nf(relation)

    if flag_1nf:
        return relation, flag_1nf
    else:
        for col in relation.columns:
            if relation[col].apply(is_list_or_set).any():
                relation = relation.explode(col)

        print('Tables after 1NF decomposition:')
        print(relation)
        print('\n')
        return relation, flag_1nf


def validate_second_nf(relation, primary_key, dependencies):
    relations = {}
    original_relation = relation
    flag_2nf = check_2nf(primary_key, dependencies)

    if flag_2nf:
        relations[primary_key] = relation
        return relations, flag_2nf
    else:
        print('Tables after 2NF decomposition:')
        for left_hand_side, right_hand_side in dependencies.items():
            cols = list(left_hand_side) + right_hand_side
            relations[tuple(left_hand_side)] = relation[cols].drop_duplicates(
            ).reset_index(drop=True)
            print(relations[left_hand_side])
            print('\n')

        junction_cols = []
        relation_name = ''
        for relation in relations:
            if set(relation).issubset(primary_key):
                relation_name += "_".join(relation)
                junction_cols.append(relation)

        if len(junction_cols) > 1:
            jun_cols = list(junction_cols)
            cols = [element for tup in jun_cols for element in tup]
            temp_df = original_relation[cols].drop_duplicates(
            ).reset_index(drop=True)

            renamed_cols = [col + '_fk' for col in cols]
            temp_df.columns = renamed_cols + \
                [col for col in temp_df.columns if col not in cols]

            temp_df[relation_name] = range(1, len(temp_df) + 1)
            columns_order = [relation_name] + renamed_cols
            temp_df = temp_df[columns_order]
            relations[relation_name] = temp_df
            print(relations[relation_name])
            print('\n')

        return relations, flag_2nf


def validate_third_nf(relations, primary_key, dependencies):
    three_relations = {}
    flag_3nf = check_3nf(relations, dependencies)

    if flag_3nf:
        return relations, flag_3nf
    else:
        print('Tables after 3NF decomposition:')
        for relation in relations:
            original_relation = relations[relation]
            for left_hand_side, right_hand_side in dependencies.items():
                cols = list(left_hand_side) + right_hand_side
                three_relations[tuple(left_hand_side)] = relations[relation][cols].drop_duplicates(
                ).reset_index(drop=True)
                print(three_relations[left_hand_side])
                print('\n')

        junction_cols = []
        relation_name = ''
        for relation in three_relations:
            relation_name += "_".join(relation)
            junction_cols.append(relation)

        print(relation_name)

        if len(junction_cols) > 1:
            jun_cols = list(junction_cols)
            cols = [element for tup in jun_cols for element in tup]
            temp_df = original_relation[cols].drop_duplicates(
            ).reset_index(drop=True)

            renamed_cols = [col + '_fk' for col in cols]
            temp_df.columns = renamed_cols + \
                [col for col in temp_df.columns if col not in cols]

            temp_df[relation_name] = range(1, len(temp_df) + 1)
            columns_order = [relation_name] + renamed_cols
            temp_df = temp_df[columns_order]
            three_relations[relation_name] = temp_df
            print(three_relations[relation_name])
            print('\n')

        return three_relations, flag_3nf


def validate_bc_nf(relations, primary_key, dependencies):
    relations = list(relations.values())
    bcnf_relations = []
    flag_bcnf = check_bcnf(relations, primary_key, dependencies)

    if flag_bcnf:
        return relations, flag_bcnf
    else:
        print('Tables after BCNF decomposition:')
        for relation in relations:
            bcnf_decomposed_relation = bcnf_decomposition(
                relation, dependencies)
            if len(bcnf_decomposed_relation) == 1:
                bcnf_relations.append(bcnf_decomposed_relation)
            else:
                relations.extend(bcnf_decomposed_relation)

    return bcnf_relations, flag_bcnf


def validate_fourth_nf(relations, mvd_dependencies):
    four_relations = []
    flag_4nf = check_4nf(relations, mvd_dependencies)

    if flag_4nf:
        return relations, flag_4nf
    else:
        print('Tables after 4NF decomposition:')
        for relation in relations:
            for left_hand_side, right_hand_sides in mvd_dependencies.items():
                for right_hand_side in right_hand_sides:
                    if isinstance(left_hand_side, tuple):
                        left_hand_side_cols = list(left_hand_side)
                    else:
                        left_hand_side_cols = [left_hand_side]

                    if all(col in relation.columns for col in left_hand_side_cols + [right_hand_side]):
                        # Check for multi-valued dependency
                        grouped = relation.groupby(left_hand_side_cols)[
                            right_hand_side].apply(set).reset_index()
                        if len(grouped) < len(relation):
                            table_1 = relation[left_hand_side_cols +
                                               [right_hand_side]].drop_duplicates()
                            table_2 = relation[left_hand_side_cols + [col for col in relation.columns if col not in [
                                right_hand_side] + left_hand_side_cols]].drop_duplicates()

                            four_relations.extend([table_1, table_2])

                            break
                else:
                    continue
                break
            else:
                four_relations.append(relation)

    if len(four_relations) == len(relations):
        return four_relations
    else:
        return validate_fourth_nf(four_relations, mvd_dependencies)


def decompose_5nf(dataframe, candidate_keys):
    def project(df, attributes):
        return df[list(attributes)].drop_duplicates().reset_index(drop=True)

    # Function to check if a decomposition is lossless
    def is_lossless(df, df1, df2):
        common_columns = set(df1.columns) & set(df2.columns)
        if not common_columns:
            return False
        joined_df = pd.merge(df1, df2, how='inner', on=list(common_columns))
        return df.equals(joined_df)

    decomposed_tables = [dataframe]

    for key in candidate_keys:
        new_tables = []
        for table in decomposed_tables:
            if set(key).issubset(set(table.columns)):
                table1 = project(table, key)
                remaining_columns = set(table.columns) - set(key)
                table2 = project(table, remaining_columns | set(key))

                if is_lossless(table, table1, table2):
                    new_tables.extend([table1, table2])
                else:
                    new_tables.append(table)
            else:
                new_tables.append(table)
        decomposed_tables = new_tables

    return decomposed_tables


def validate_fifth_nf(relations, primary_key, dependencies):
    five_relations = []
    flag_5nf, candidate_keys_dict = check_5nf(relations)

    if flag_5nf:
        return relations, flag_5nf
    else:
        print('Tables after 5NF decomposition:')
        i = 0
        for relation in relations:
            candidate_keys = candidate_keys_dict[i]
            i += 1
            decomposed_relations = decompose_5nf(relation, candidate_keys)
            five_relations.append(decomposed_relations)

    return five_relations, flag_5nf


In [9]:
# This file is used for output generator
def pd2sql(dtype):
    """Function to convert pandas dtype to SQL data type."""
    if "int" in str(dtype):
        return "INT"
    elif "float" in str(dtype):
        return "FLOAT"
    elif "object" in str(dtype):
        return "VARCHAR(255)"
    elif "datetime" in str(dtype):
        return "DATETIME"
    else:
        return "TEXT"


def generate_1nf(primary_keys, df):
    table_name = "_".join(primary_keys) + "_table"
    # Start creating the SQL query
    query = f"CREATE TABLE {table_name} (\n"

    for column, dtype in zip(df.columns, df.dtypes):
        if column in primary_keys:
            query += f"  {column} {pd2sql(dtype)} PRIMARY KEY,\n"
        else:
            query += f"  {column} {pd2sql(dtype)},\n"

    query = query.rstrip(',\n') + "\n);"

    print(query)


def generate_2nf_3nf(relations):
    for relation in relations:
        primary_keys = relation
        table_name = "_".join(relation) + '_table'
        relation = relations[relation]

        query = f"CREATE TABLE {table_name} (\n"

        for column, dtype in zip(relation.columns, relation.dtypes):
            if column in primary_keys:
                query += f"  {column} {pd2sql(dtype)} PRIMARY KEY,\n"
            else:
                query += f"  {column} {pd2sql(dtype)},\n"

        query = query.rstrip(',\n') + "\n);"

        print(query)


def generate_bcnf_4nf_5nf(relations):
    for relation in relations:
        primary_key = relation.columns[0]
        table_name = f'{primary_key}_table'

        query = f"CREATE TABLE {table_name} (\n"

        for column, dtype in zip(relation.columns, relation.dtypes):
            if column == primary_key:
                query += f"  {column} {pd2sql(dtype)} PRIMARY KEY,\n"
            else:
                query += f"  {column} {pd2sql(dtype)},\n"

        query = query.rstrip(',\n') + "\n);"

        print(query)


In [4]:
# This is the main file for reading CSV data and performing normalization operations
import pandas as pd
import csv
import normalization_procedures
import input_parser
from sql_table_creator import generate_1nf, generate_2nf_3nf, generate_bcnf_4nf_5nf

# Reading the input csv file and the dependencies text file
input_file = pd.read_csv('/content/exampleInputTable.csv')
print('Input Relation Tables:')
print(input_file)
print('\n')

with open('/content/dependency_parser.txt', 'r') as file:
    lines = [line.strip() for line in file]

dependencies = {}
for line in lines:
    left_hand_side, right_hand_side = line.split(" -> ")
    left_hand_side = left_hand_side.split(", ")
    dependencies[tuple(left_hand_side)] = right_hand_side.split(", ")
print('Dependencies:')
print(dependencies)
print('\n')

# Input from the user
target_normal_form = input(
    'Choice of the highest normal form to reach (1: 1NF, 2: 2NF, 3: 3NF, B: BCNF, 4: 4NF, 5: 5NF):')
if target_normal_form in ["1", "2", "3", "4", "5"]:
    target_normal_form = int(target_normal_form)

# Find the highest normal form of the input relation
find_high_nf = int(
    input('Find the highest normal form of the input table? (1: Yes, 2: No):'))
high_nf = 'No normalization done.'

primary_key = input(
    "Enter the Primary Key values: ").split(', ')
print('\n')

keys = ()
for key in primary_key:
    keys = keys + (key,)

primary_key = keys

mvd_dependencies = {}
if not target_normal_form == 'B' and target_normal_form >= 4:
    with open('/content/mvd_dependencies.txt', 'r') as file:
        mvd_lines = [line.strip() for line in file]

    print(mvd_lines)

    for mvd in mvd_lines:
        left_hand_side, right_hand_side = mvd.split(" ->-> ")
        left_hand_side = left_hand_side.split(
            ", ") if ", " in left_hand_side else [left_hand_side]
        left_hand_side_str = str(left_hand_side)
        if left_hand_side_str in mvd_dependencies:
            mvd_dependencies[left_hand_side_str].append(right_hand_side)
        else:
            mvd_dependencies[left_hand_side_str] = [right_hand_side]

    print('Multi-valued Dependencies')
    print(mvd_dependencies)
    print('\n')

input_file = input_parser.input_parser(input_file)

if target_normal_form == 'B' or target_normal_form >= 1:
    first_nf_table, flag_1nf = normalization_procedures.validate_first_nf(
        input_file)

    if flag_1nf:
        high_nf = 'Highest Normal Form is: 1NF'

    if target_normal_form == 1:
        if flag_1nf:
            print('Already Normalized to 1NF')
            print('\n')

        print('Queries after decomposing to 1NF:')
        print('\n')
        generate_1nf(primary_key, first_nf_table)

if target_normal_form == 'B' or target_normal_form >= 2:
    second_nf_tables, flag_2nf = normalization_procedures.validate_second_nf(
        first_nf_table, primary_key, dependencies)

    if flag_1nf and flag_2nf:
        high_nf = 'Highest Normal Form is: 2NF'

    if target_normal_form == 2:
        if flag_2nf and flag_1nf:
            print('Already Normalized to 2NF')
            print('\n')

        print('Queries after decomposing to 2NF')
        print('\n')
        generate_2nf_3nf(second_nf_tables)

if target_normal_form == 'B' or target_normal_form >= 3:
    third_nf_tables, flag_3nf = normalization_procedures.validate_third_nf(
        second_nf_tables, primary_key, dependencies)

    if flag_1nf and flag_2nf and flag_3nf:
        high_nf = 'Highest Normal Form is: 3NF'

    if target_normal_form == 3:
        if flag_3nf and flag_2nf and flag_1nf:
            print('Already Normalized to 3NF')
            print('\n')

        print('Queries after decomposing to 3NF')
        print('\n')
        generate_2nf_3nf(third_nf_tables)

if target_normal_form == 'B' or target_normal_form >= 4:
    bcnf_tables, flag_bcnf = normalization_procedures.validate_bc_nf(
        third_nf_tables, primary_key, dependencies)

    if flag_1nf and flag_2nf and flag_3nf and flag_bcnf:
        high_nf = 'Highest Normal Form is: BCNF'

    if target_normal_form == 'B':
        if flag_bcnf and flag_3nf and flag_2nf and flag_1nf:
            print('Already Normalized to BCNF')
            print('\n')

        print('Queries after decomposing to BCNF')
        print('\n')
        generate_bcnf_4nf_5nf(bcnf_tables)

if not target_normal_form == 'B' and target_normal_form >= 4:
    fourth_nf_tables, flag_4nf = normalization_procedures.validate_fourth_nf(
        bcnf_tables, mvd_dependencies)

    if flag_1nf and flag_2nf and flag_3nf and flag_bcnf and flag_4nf:
        high_nf = 'Highest Normal Form is: 4NF'

    if target_normal_form == 4:
        if flag_4nf and flag_bcnf and flag_3nf and flag_2nf and flag_1nf:
            print('Already Normalized to 4NF')
            print('\n')

        print('Queries after decomposing to 4NF')
        print('\n')
        generate_bcnf_4nf_5nf(fourth_nf_tables)

if not target_normal_form == 'B' and target_normal_form >= 5:
    fifth_nf_tables, flag_5nf = normalization_procedures.validate_fifth_nf(
        fourth_nf_tables, primary_key, dependencies)

    if flag_1nf and flag_2nf and flag_3nf and flag_bcnf and flag_4nf and flag_5nf:
        high_nf = 'Highest Normal Form is: 5NF'

    if target_normal_form == 5:
        if flag_5nf and flag_4nf and flag_bcnf and flag_3nf and flag_2nf and flag_1nf:
            print('Already Normalized to 5NF')
            print('\n')

        print('Queries after decomposing to 5NF')
        print('\n')
        generate_bcnf_4nf_5nf(fifth_nf_tables)

if find_high_nf == 1:
    print('\n')
    print(high_nf)
    print('\n')


Input Relation Tables:
   StudentID FirstName  LastName   Course  Professor  ProfessorEmail  \
0        101      John       Doe  Math101   Dr.Smith   smith@mst.edu   
1        102      Jane       Roe  Math101   Dr.Smith   smith@mst.edu   
2        103   Arindam    Khanda    CS101   Dr.Jones   jones@mst.edu   
3        104      Jose  Franklin   Bio101  Dr.Watson  watson@mst.edu   
4        105       Ada  Lovelace    CS101   Dr.Jones   jones@mst.edu   

  CourseStart  CourseEnd  
0  01-01-2023  5/30/2023  
1  01-01-2023  5/30/2023  
2  02-01-2023  6/15/2023  
3  03-01-2023  7/20/2023  
4  02-01-2023  6/15/2023  


Dependencies:
{('StudentID',): ['FirstName', 'LastName'], ('Course',): ['CourseStart', 'CourseEnd', 'Professor'], ('Professor',): ['ProfessorEmail']}


Choice of the highest normal form to reach (1: 1NF, 2: 2NF, 3: 3NF, B: BCNF, 4: 4NF, 5: 5NF):2
Find the highest normal form of the input table? (1: Yes, 2: No):1
Enter the Primary Key values: StudentID, Course


Tables after 2NF